## 201 - Engineering Text Features Using `mmlspark` Modules and Spark SQL

Again, try to predict Amazon book ratings greater than 3 out of 5, this time using
the `TextFeaturizer` module which is a composition of several text analytics APIs that
are native to Spark.

In [1]:
import pandas as pd


StatementMeta(SamplePool, 46, 1, Finished, Available)

In [2]:
data = spark.read.parquet("wasbs://publicwasb@mmlspark.blob.core.windows.net/BookReviewsFromAmazon10K.parquet")
data.limit(10).toPandas()

StatementMeta(SamplePool, 46, 2, Finished, Available)

   rating                                               text
0       2  Ok~ but I think the Keirsey Temperment Test is...
1       2  Repellent Sale of Conservativism  The fatalist...
2       1  I had a bad feeling about this!  And I was rig...
3       2  Lost Credability, QUICKLY!!  I admit, I haven'...
4       2  Poorly written  I tried reading this book but ...
5       2  The book felt more like the author was forced ...
6       1  The Islamo-Fascists Murderers thank Professor ...
7       2  Stranded on an Island  I have been a fan of Su...
8       2  Another self-indulgent Heinlein novel  Heinlei...
9       2  Beautiful Beginning / Painful (illogical) Endi...

Use `TextFeaturizer` to generate our features column.  We remove stop words, and use TF-IDF
to generate 2?? sparse features.

In [3]:
from mmlspark.featurize.text import TextFeaturizer
textFeaturizer = TextFeaturizer() \
  .setInputCol("text").setOutputCol("features") \
  .setUseStopWordsRemover(True).setUseIDF(True).setMinDocFreq(5).setNumFeatures(1 << 16).fit(data)

StatementMeta(SamplePool, 46, 3, Finished, Available)

In [4]:
processedData = textFeaturizer.transform(data)
processedData.limit(5).toPandas()

StatementMeta(SamplePool, 46, 4, Finished, Available)

   rating  ...                                           features
0       2  ...  (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1       2  ...  (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2       1  ...  (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3       2  ...  (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4       2  ...  (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...

[5 rows x 3 columns]
/opt/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py:2110: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)

Change the label so that we can predict whether the rating is greater than 3 using a binary
classifier.

In [5]:
processedData = processedData.withColumn("label", processedData["rating"] > 3) \
                             .select(["features", "label"])
processedData.limit(5).toPandas()

StatementMeta(SamplePool, 46, 5, Finished, Available)

                                            features  label
0  (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  False
1  (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  False
2  (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  False
3  (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  False
4  (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  False

Train several Logistic Regression models with different regularizations.

In [6]:
train, test, validation = processedData.randomSplit([0.60, 0.20, 0.20])
from pyspark.ml.classification import LogisticRegression

lrHyperParams = [0.05, 0.1, 0.2, 0.4]
logisticRegressions = [LogisticRegression(regParam = hyperParam) for hyperParam in lrHyperParams]

from mmlspark.train import TrainClassifier
lrmodels = [TrainClassifier(model=lrm, labelCol="label").fit(train) for lrm in logisticRegressions]

StatementMeta(SamplePool, 46, 6, Finished, Available)

Find the model with the best AUC on the test set.

In [7]:
from mmlspark.automl import FindBestModel, BestModel
bestModel = FindBestModel(evaluationMetric="AUC", models=lrmodels).fit(test)
bestModel.getEvaluationResults().show()
bestModel.getBestModelMetrics().show()
bestModel.getAllModelMetrics().show()


StatementMeta(SamplePool, 46, 7, Finished, Available)

+-------------------+--------------------+
|false_positive_rate|  true_positive_rate|
+-------------------+--------------------+
|                0.0|                 0.0|
|                0.0|6.253908692933083E-4|
|                0.0|0.001250781738586...|
|                0.0|0.001876172607879925|
|                0.0|0.002501563477173...|
|                0.0|0.003126954346466...|
|                0.0| 0.00375234521575985|
|                0.0|0.004377736085053158|
|                0.0|0.005003126954346...|
|                0.0|0.005628517823639775|
|                0.0|0.006253908692933083|
|                0.0|0.006879299562226...|
|                0.0|0.008130081300813009|
|                0.0|0.008755472170106316|
|                0.0|0.009380863039399626|
|                0.0|0.010006253908692933|
|                0.0|0.010631644777986242|
|                0.0| 0.01125703564727955|
|                0.0|0.011882426516572859|
|                0.0|0.012507817385866166|
+----------

Use the optimized `ComputeModelStatistics` API to find the model accuracy.

In [8]:
from mmlspark.train import ComputeModelStatistics
predictions = bestModel.transform(validation)
metrics = ComputeModelStatistics().transform(predictions)
print("Best model's accuracy on validation set = "
      + "{0:.2f}%".format(metrics.first()["accuracy"] * 100))

StatementMeta(SamplePool, 46, 8, Finished, Available)

Best model's accuracy on validation set = 82.62%

In [9]:
spark.stop()

StatementMeta(SamplePool, 46, 9, Finished, Available)